In [11]:
import requests
from connection import ConnectionTask as ConnectTask
import time
import hashlib
import json
from pprint import pprint
c=ConnectTask()

In [12]:
# 测试环境地址：http://movie.test.maoyan.com
# 渠道id：1000014
# 渠道key：A013F70DB97834C0A5492378BD76C53A
# ID="1000014"
# KEY="A013F70DB97834C0A5492378BD76C53A"
# url_head="http://movie.test.maoyan.com"
ID = "1000058"
KEY = "cw8jhcd1lld0b5c3szq890wgxrq376cc"
url_head = "http://open.maoyan.com/api/"

### 2.1 查询影院列表 s

In [13]:
url_tail="/sync/gateway"
api_name="gateway.sync.cinemas"
data={
    "merCode":ID,
     "timestamp":int(time.time()),
     "version":"1.0",
     "signType":"MD5",
     "api":api_name}
md5_str="".join([k+"="+str(v)+"&" for k,v in sorted(data.items())])+"key="+KEY
sign=hashlib.md5(md5_str.encode('utf-8')).hexdigest().upper()
data["signMsg"]=sign

resp_data=requests.post(url=url_head+url_tail,data=data).json()
# resp_data
# pprint(resp_data)
# for item in resp_data['data']['data']:
#     cinema_id=item.pop("cinemaId")
#     c.dev.dianying_cinema_mao.update_one({"cinemaId":cinema_id},
#                                  {"$set":item},upsert=True)

In [14]:
resp_data['data']['data'][0]

{'areaName': '花山区',
 'buyNumLimit': 4,
 'buyTimeLimit': 15,
 'cinemaAddr': '湖南东路与霍里山大道交叉口',
 'cinemaId': 25201,
 'cinemaName': '恒大影城',
 'cityId': '194',
 'cityName': '马鞍山',
 'dis': '',
 'districtId': '2898',
 'features': [],
 'latitude': '31.68464',
 'locationId': '340503',
 'longitude': '118.5553',
 'note': '',
 'stdCinemaCode': '34052901',
 'subway': '',
 'supportQR': 1,
 'takeTicketPlace': '影院内自助取票机或影院前台',
 'tel': '0555-2347888'}

In [15]:
len(resp_data['data']['data'])
[item for item in resp_data['data']['data'] if item['cityName'] =='奉化']

[]

In [6]:
maoyan_citys = {item['cityName']:item['cityId'] for item in resp_data['data']['data']}
with open('/data/logs/maoyan_citys.json','w')as f:
    json.dump(maoyan_citys,f)

In [11]:
import tablib
tab_data = tablib.Dataset(headers=["城市名称","城市ID"])
for city_name,city_id in maoyan_citys.items():
    tab_data.append([city_name,city_id])
with open("/root/notebook/Master/猫眼支持城市列表.xlsx","wb") as f:
    f.write(tab_data.xlsx)

/usr/local/python3/lib/python3.6/site-packages/openpyxl/worksheet/worksheet.py:303: UserWarning: Using a coordinate with ws.cell is deprecated. Use ws[coordinate] instead
  warn("Using a coordinate with ws.cell is deprecated. Use ws[coordinate] instead")


In [12]:
len(maoyan_citys)

330

### 2.2 查询影厅列表

In [ ]:
cinema_id=11
url_tail="/sync/gateway"
api_name="gateway.sync.halls"
data={
    "merCode":ID,
     "timestamp":int(time.time()),
     "version":"1.0",
     "signType":"MD5",
     "api":api_name,
     "bizData":json.dumps({"cinemaId":cinema_id})}
md5_str="".join([k+"="+str(v)+"&" for k,v in sorted(data.items())])+"key="+KEY
sign=hashlib.md5(md5_str.encode('utf-8')).hexdigest().upper()
data["signMsg"]=sign

requests.post(url=url_head+url_tail,data=data).json()

### 2.3 查询影厅座位列表

In [ ]:
cinema_id=11
hall_id=4427
url_tail="/sync/gateway"
api_name="gateway.sync.seats"
data={
    "merCode":ID,
     "timestamp":int(time.time()),
     "version":"1.0",
     "signType":"MD5",
     "api":api_name,
     "bizData":json.dumps({"cinemaId":cinema_id,"hallId":hall_id})}
md5_str="".join([k+"="+str(v)+"&" for k,v in sorted(data.items())])+"key="+KEY
sign=hashlib.md5(md5_str.encode('utf-8')).hexdigest().upper()
data["signMsg"]=sign

requests.post(url=url_head+url_tail,data=data).json()

In [ ]:
from collections import Counter
city_id_list = c.static_cluster.dianying_cinema_mao.distinct("cityId")
city_id_cursor =[item['cityId'] for item in c.static_cluster.dianying_cinema_mao.find({},{"cityId":1})]
counter = Counter(city_id_cursor)
pprint(counter)

In [ ]:
Counter([1])

### 2.4 查询场次列表y

In [10]:
cinema_id=1
start_date="2018-03-09"
url_tail="/sync/gateway"
api_name="gateway.sync.show"
data={
    "merCode":ID,
     "timestamp":int(time.time()),
     "version":"1.0",
     "signType":"MD5",
     "api":api_name,
     "bizData":json.dumps({"cinemaId":cinema_id,"startDate":start_date})}
md5_str="".join([k+"="+str(v)+"&" for k,v in sorted(data.items())])+"key="+KEY
sign=hashlib.md5(md5_str.encode('utf-8')).hexdigest().upper()
data["signMsg"]=sign


resp_data=requests.post(url=url_head+url_tail,data=data).json()
pprint(resp_data)
# document={}
# for item in resp_data['data']['data']:
#     cinema_id=item.pop("cinemaId")
#     film_id=item.pop("filmId")
#     document.setdefault(film_id,[])
#     document[film_id].append(item)

# for k,v in document.items():
#     c.dev.dianying_film_mao.update_one({"cinemaId":cinema_id,"id":k},
#                                  {"$set":{"screenings":v}},upsert=True)
#     query_the_movie_information(k)

{'api': 'gateway.sync.show',
 'code': '0',
 'data': {'bizCode': '0',
          'bizMsg': 'SUCCESS',
          'data': [{'canSell': True,
                    'cinemaId': 1,
                    'fee': 300,
                    'filmId': 1217637,
                    'filmName': '厉害了，我的国',
                    'hallCode': '0000000000000007',
                    'hallName': '8号激光厅',
                    'language': '国语',
                    'minPrice': 3000,
                    'movieStandId': None,
                    'price': 7000,
                    'settlePriceForCinema': 3200,
                    'settlePriceForMerchant': 3500,
                    'showId': '201803130004156',
                    'showScreenType': '普通',
                    'showStartTime': 1520926200000,
                    'showVersionType': '2D'},
                   {'canSell': True,
                    'cinemaId': 1,
                    'fee': 300,
                    'filmId': 1217637,
                    'filmName': 

                    'settlePriceForCinema': 3700,
                    'settlePriceForMerchant': 4000,
                    'showId': '201803090192236',
                    'showScreenType': '普通',
                    'showStartTime': 1520592900000,
                    'showVersionType': '3D'},
                   {'canSell': True,
                    'cinemaId': 1,
                    'fee': 300,
                    'filmId': 1182552,
                    'filmName': '红海行动',
                    'hallCode': '0000000000000004',
                    'hallName': '5号厅',
                    'language': '国语',
                    'minPrice': 3500,
                    'movieStandId': '00100012018',
                    'price': 9000,
                    'settlePriceForCinema': 3700,
                    'settlePriceForMerchant': 4000,
                    'showId': '201803090192235',
                    'showScreenType': '普通',
                    'showStartTime': 1520594100000,
                    'sho

In [35]:
cache_hot = c.dpprdb(0).smembers("dpp:maoyan.cache:hot:10")

In [32]:
c.dpprdb(0).srem("dpp:maoyan.cache:hot:10", *value)

6

In [30]:
value={'1132373','1132973','1136547','1142586','1157354','1170264'}

In [37]:
c.dpprdb(0).sadd("dpp:maoyan.cache:hot:10", *cache_hot)

0

In [36]:
cache_hot

{'1132373',
 '1132973',
 '1136547',
 '1142586',
 '1157354',
 '1170264',
 '1177097',
 '1177952',
 '1182552',
 '1189089',
 '1189346',
 '1190357',
 '1197732',
 '1199446',
 '1199974',
 '1200609',
 '1200984',
 '1201381',
 '1203436',
 '1203620',
 '1203768',
 '1204714',
 '1204720',
 '1204800',
 '1205145',
 '1207099',
 '1208113',
 '1208122',
 '1208343',
 '1208748',
 '1209260',
 '1210930',
 '1211063',
 '1211086',
 '1211465',
 '1211482',
 '1211538',
 '1211570',
 '1211693',
 '1212450',
 '1212533',
 '1212772',
 '1212787',
 '1212979',
 '1212982',
 '1213016',
 '1213057',
 '1213080',
 '1213224',
 '1213237',
 '1213242',
 '1213724',
 '1213729',
 '1217159',
 '13457',
 '246083',
 '246100',
 '246131',
 '246371',
 '246714',
 '247083',
 '247948',
 '248639',
 '306969',
 '329733',
 '337226',
 '337443',
 '337511',
 '338463',
 '338493',
 '340946',
 '341084',
 '341138',
 '341323',
 '341540',
 '341897',
 '341999',
 '342754',
 '343136',
 '343526',
 '343762',
 '343887',
 '343905',
 '343944',
 '343945',
 '344080',
 

### 2.6锁座

In [ ]:
def lock_seat(cinema_id,show_id):
    url_tail="/trade/gateway"
    api_name="gateway.trade.lock"
    data={
        "merCode":ID,
         "timestamp":int(time.time()),
         "version":"1.0",
         "signType":"MD5",
         "api":api_name,
         "bizData":json.dumps({"cinemaId":cinema_id,
                              "showId":show_id,
                              "seatsJSON":'{"count":"2","list":[{"sectionId":"01","seatNo":"A_10","columnId":"01","rowId":"A"},{"sectionId":"01","seatNo":"A_09","columnId":"02","rowId":"A"}]}',
                               "orderCode":"blm10086",
                               "mobile":"1008611",
                               "settlePrice":10000,
                               "sellPrice":10000
                              })}
    md5_str="".join([k+"="+str(v)+"&" for k,v in sorted(data.items())])+"key="+KEY
    sign=hashlib.md5(md5_str.encode('utf-8')).hexdigest().upper()
    data["signMsg"]=sign

    resp_data=requests.post(url=url_head+url_tail,data=data).json()
    pprint(resp_data)
lock_seat(11,201707250000050)

### 2.8确认订单

In [ ]:
def confirm_order(order_id):
    url_tail="/trade/gateway"
    api_name="gateway.trade.fixOrder"
    data={
        "merCode":ID,
         "timestamp":int(time.time()),
         "version":"1.0",
         "signType":"MD5",
         "api":api_name,
         "bizData":json.dumps({"orderId":order_id})}
    md5_str="".join([k+"="+str(v)+"&" for k,v in sorted(data.items())])+"key="+KEY
    sign=hashlib.md5(md5_str.encode('utf-8')).hexdigest().upper()
    data["signMsg"]=sign

    resp_data=requests.post(url=url_head+url_tail,data=data).json()
    pprint(resp_data)
query_the_order_information(69262689)

### 2.11查询订单信息

In [3]:
def query_the_order_information(order_id):
    url_tail="/trade/gateway"
    api_name="gateway.trade.queryOrder"
    data={
        "merCode":ID,
         "timestamp":int(time.time()),
         "version":"1.0",
         "signType":"MD5",
         "api":api_name,
         "bizData":json.dumps({"orderId":order_id})}
    md5_str="".join([k+"="+str(v)+"&" for k,v in sorted(data.items())])+"key="+KEY
    sign=hashlib.md5(md5_str.encode('utf-8')).hexdigest().upper()
    data["signMsg"]=sign

    resp_data=requests.post(url=url_head+url_tail,data=data).json()
    pprint(resp_data)
query_the_order_information(71150649)

{'api': 'gateway.trade.queryOrder',
 'code': '0',
 'data': {'bizCode': '0',
          'bizMsg': 'SUCCESS',
          'data': {'cinemaId': 12769,
                   'exchangeTime': None,
                   'fixFailTime': None,
                   'fixSucTime': 1518775910000,
                   'hallId': 141461,
                   'lockTime': 1518775826851,
                   'orderCode': 'bp203643778805406',
                   'orderId': 71150649,
                   'orderStatus': '5',
                   'qrCode': '2018021759021365',
                   'seatsJSON': '{"count":1,"list":[{"sectionId":"0000000000000001","seatNo":"0000000000000001-4-06","columnId":"06","rowId":"4"}]}',
                   'sellPrice': '5900',
                   'settlePrice': '5900',
                   'showId': '201802170186888',
                   'ticketCode': '2018021759021365',
                   'ticketCodeName': '验证码'}},
 'msg': 'SUCCESS',
 'version': '1.0'}


### 2.12 查询电影信息

In [ ]:
def query_the_movie_information(movie_id):
#     url_tail="/mmdb/gateway"
#     api_name="mmdb.movieInfo"
    url_tail = "/cmdata/gateway"
    api_name = "cmdata.movieInfo"
    data={
        "merCode":ID,
         "timestamp":int(time.time()),
         "version":"1.0",
         "signType":"MD5",
         "api":api_name,
         "bizData":json.dumps({"movieId":movie_id})}
    md5_str="".join([k+"="+str(v)+"&" for k,v in sorted(data.items())])+"key="+KEY
    sign=hashlib.md5(md5_str.encode('utf-8')).hexdigest().upper()
    data["signMsg"]=sign

    resp_data=requests.post(url=url_head+url_tail,data=data).json()
    pprint(resp_data)
#     item=resp_data['data']['data']
#     film_id=item.pop("id")
#     c.dev.dianying_film_mao.update_many({"id":film_id},{"$set":item})

In [ ]:
query_the_movie_information(1197732)

In [ ]:
c.static_c

### 2.13 查询热映电影列表

In [ ]:
city_id=1

url_tail="/mmdb/gateway"
api_name="mmdb.hotMovies"
data={
    "merCode":ID,
     "timestamp":int(time.time()),
     "version":"1.0",
     "signType":"MD5",
     "api":api_name,
     "bizData":json.dumps({"cityId":city_id})}
md5_str="".join([k+"="+str(v)+"&" for k,v in sorted(data.items())])+"key="+KEY
sign=hashlib.md5(md5_str.encode('utf-8')).hexdigest().upper()
data["signMsg"]=sign

requests.post(url=url_head+url_tail,data=data).json()

### 2.14 查询待映电影列表

In [ ]:
city_id=1

url_tail="/mmdb/gateway"
api_name="mmdb.comingMovies"
data={
    "merCode":ID,
     "timestamp":int(time.time()),
     "version":"1.0",
     "signType":"MD5",
     "api":api_name,
     "bizData":json.dumps({"cityId":city_id})}
md5_str="".join([k+"="+str(v)+"&" for k,v in sorted(data.items())])+"key="+KEY
sign=hashlib.md5(md5_str.encode('utf-8')).hexdigest().upper()
data["signMsg"]=sign

requests.post(url=url_head+url_tail,data=data).json()

### 2.15 查询电影热评列表

In [ ]:
movie_id=248911

url_tail="/mmdb/gateway"
api_name="mmdb.hotComments"
data={
    "merCode":ID,
     "timestamp":int(time.time()),
     "version":"1.0",
     "signType":"MD5",
     "api":api_name,
     "bizData":json.dumps({"movieId":movie_id})}
md5_str="".join([k+"="+str(v)+"&" for k,v in sorted(data.items())])+"key="+KEY
sign=hashlib.md5(md5_str.encode('utf-8')).hexdigest().upper()
data["signMsg"]=sign

requests.post(url=url_head+url_tail,data=data).json()

### 2.16 查询城市列表

In [ ]:
url_tail="/base/gateway"
api_name="base.city"
data={
    "merCode":ID,
     "timestamp":int(time.time()),
     "version":"1.0",
     "signType":"MD5",
     "api":api_name}
md5_str="".join([k+"="+str(v)+"&" for k,v in sorted(data.items())])+"key="+KEY
sign=hashlib.md5(md5_str.encode('utf-8')).hexdigest().upper()
data["signMsg"]=sign
resp = requests.post(url=url_head+url_tail,data=data).json()
resp

In [ ]:
city1={item['name']:item['id'] for item in resp['data']['data']}

In [ ]:
maoyan = requests.get("http://mongorestapi.localdomain:8024/query/citys/dianying/maoyan").json()
city2={item['name']:item['channelCityId'] for item in maoyan['data']}

In [ ]:
for city in city2:
    if city in city1:
        if city2[city] != city1[city]:
            print(city,city2[city],city1[city])

In [ ]:
for item in c.static.dianying_cinema_mao.distinct('cinemaId').extend(c.static.dianying_film_mao.distinct('cinemaId')):
    print(item)
    break

In [ ]:
for item in set(c.static.dianying_cinema_mao.distinct('cinemaId')+c.static.dianying_film_mao.distinct('cinemaId')):
    print(item)
    break

In [ ]:
def filter_dict(coll, filter_value=None):
    if not isinstance(coll, dict):
        return coll
    if not isinstance(filter_value, list):
        filter_value = [filter_value]
    return {key: value for key, value in coll.items() if value not in filter_value}
filter_dict({'a':'','b':None,'c':1},filter_value=[None,''])

In [ ]:
%%timeit
a=list(range(1000))
b=list(range(100,1100))
set(a+b)

In [ ]:
%%timeit
a=list(range(1000))
b=list(range(100,1100))
a.extend(b)
set(a)

In [ ]:
%%timeit
a=set(list(range(1000)))
b=list(range(100,1100))
a.update(set(b))